In [ ]:
!pip install transformers torch nltk datasets Pillow


In [ ]:
from transformers import VisionEncoderDecoderModel,ViTFeatureExtractor, ViTImageProcessor, AutoTokenizer,  BertTokenizer, BertForSequenceClassification
import torch
from PIL import Image
from nltk.translate.bleu_score import corpus_bleu
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from datasets import load_dataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW


# Constants
max_length = 15
num_beams = 6

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


dataset = load_dataset("biglam/dating-historical-color-images")

decade_labels = {
    0: '1930',
    1: '1940',
    2: '1950',
    3: '1960',
    4: '1970'
}


class HistoricalImageDataset(Dataset):
    def __init__(self, dataset, captions= None, transform=None):
        self.dataset = dataset
        self.captions = captions
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image'].convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Append the date to the pre-generated caption
        date = decade_labels[item['label']]
        full_caption = self.captions[idx].rstrip() + f' during {date}'

        return image, full_caption




# Transformations for the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
])

# Create datasets and data loaders
train_dataset = HistoricalImageDataset(dataset=dataset['train'], transform=transform)

# Create DataLoader
data_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)




In [ ]:

optimizer = AdamW(model.parameters(), lr=4e-5)
num_epochs = 5
criterion = CrossEntropyLoss()
print_interval = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i, (images, captions) in enumerate(data_loader):
        images = images.to(device)
        inputs = tokenizer(captions, padding=True, max_length=max_length, return_tensors="pt")
        input_ids = inputs.input_ids.to(device)

        # Prepare shifted labels for the decoder
        labels = torch.roll(input_ids, -1, dims=1)
        labels = labels.to(device)

        optimizer.zero_grad()
        pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values.to(device)
        outputs = model(pixel_values=pixel_values, labels=labels)

        # Check and compute loss
        loss = outputs.loss if outputs.loss is not None else criterion(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % print_interval == 0:
            # Print caption every 'print_interval' iterations
            print("Caption:", captions)
            print(f"Iteration [{i + 1}/{len(data_loader)}], Epoch {epoch + 1}/{num_epochs}, Total Loss: {total_loss}")


    print(f"Epoch {epoch + 1}/{num_epochs}, Total Loss: {total_loss}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Caption: ('a mountain range with a river and mountains during 1950', 'a large building with a clock on top of it during 1940')
Iteration [100/663], Epoch 1/5, Total Loss: 219.33620190620422
Caption: ('a man standing on top of a hill holding a stick during 1930', 'a double decker bus parked in front of a brick building during 1970')
Iteration [200/663], Epoch 1/5, Total Loss: 387.8956495523453
Caption: ('a beach filled with lots of sand and water during 1970', 'a large group of people standing in front of a sign during 1960')
Iteration [300/663], Epoch 1/5, Total Loss: 542.3875263929367
Caption: ('two little girls standing next to each other during 1950', 'a man standing in front of a truck during 1940')
Iteration [400/663], Epoch 1/5, Total Loss: 692.8588055968285
Caption: ('a train on a train track during 1930', 'a large building with a clock on the front of it during 1950')
Iteration [500/663], Epoch 1/5, Total Loss: 836.9095251560211
Caption: ('a man standing next to an old fashione

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from nltk.translate.bleu_score import corpus_bleu
import nltk
nltk.download('punkt')


def generate_caption_for_url_image(model, feature_extractor, url, max_length=16, num_beams=4):
    """
    Generate a caption for an image obtained from a URL.

    Parameters:
    model: The loaded VisionEncoderDecoderModel.
    feature_extractor: The loaded feature extractor for the model.
    url (str): URL of the image.
    max_length (int): Maximum length of the generated caption.
    num_beams (int): Number of beams for beam search.

    Returns:
    str: Generated caption for the image.
    """
    # Download the image from the URL
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))

    # Process the image
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(model.device)

    # Generate caption
    model.eval()
    with torch.no_grad():
        output_ids = model.generate(pixel_values, max_length=max_length, num_beams=num_beams)

    # Decode the generated caption
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcShbm3S6hBp1pfAM43tpHpdyf_8Mgvx1-MzQlIGp2xFFQ&s'
generated_caption = generate_caption_for_url_image(model, feature_extractor, url)
real_caption = 'a car parked in front of a building during 1950'
print(generated_caption)
real_tokens = nltk.word_tokenize(real_caption.lower())
generated_tokens = nltk.word_tokenize(generated_caption.lower())


real_caption_list = [real_tokens]
generated_caption_list = [generated_tokens]

# Calculate BLEU score
bleu_score = corpus_bleu([real_caption], [generated_caption], weights = (1,0,0,0))
print(f"BLEU score: {bleu_score}")


file_path = "saved_images.txt"

with open(file_path, "w") as file:
    for real_sent, generated_sent in zip(real_caption_list, generated_caption_list):
        real_sent_str = " ".join(real_sent)
        generated_sent_str = " ".join(generated_sent)

        file.write(f"Real Caption: {real_sent_str}\n")
        file.write(f"Generated Caption: {generated_sent_str}\n")

print(f"Captions saved to {file_path}")


In [ ]:
count = 0

def numbers_match(string1, string2):
    """
    Check if any numbers present in string1 are also present in string2.

    Args:
    string1 (str): The first string to be analyzed.
    string2 (str): The second string to be analyzed.

    Returns:
    bool: True if any number in string1 is found in string2, False otherwise.
    """
    # Extracting numbers from the first string
    numbers_in_string1 = set(filter(str.isdigit, string1))

    # Checking if any number from string1 exists in string2
    for number in numbers_in_string1:
        if number in string2:
            return True

    return False

import os

directory_path = "saved_images"

count = 0

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    with open(file_path, "r") as file:
        real_caption = file.readline().strip()
        generated_caption = file.readline().strip()

    if numbers_match(real_caption, generated_caption):
        print("Success!")
        count += 1
    else:
        print("No Match!")
        count += 1

print(f"Total matches found: {count}")

In [ ]:
# Calculate the F1 Score

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Compute the confusion matrix
confusion_matrix = confusion_matrix(real_caption_list, generated_caption_list)


import numpy as np


# Calculate F1 score for each class
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print F1 scores for each class
for i, date_range in enumerate(["1930s", "1940s", "1950s", "1960s", "1970s"]):
    print(f"F1 Score ({date_range}): {f1_score[i]:.2f}")

# Calculate weighted average F1 score
weights = np.sum(confusion_matrix, axis=1) / np.sum(confusion_matrix)
weighted_average_f1_score = np.sum(f1_score * weights)

print(f"Weighted Average F1 Score: {weighted_average_f1_score:.2f}")
